In [ ]:
# Import local bentoml repository
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), "..", ".."))
import bentoml

print(bentoml.__file__)

# Titanic Survival Prediction.

Package and deploy with BentoML

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import re

## Dataset
download dataset from https://www.kaggle.com/c/titanic/data

In [ ]:
!mkdir data
!curl https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/train.csv -o ./data/train.csv
!curl https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/test.csv -o ./data/test.csv

In [ ]:
train = pd.read_csv("./data/train.csv")
test  = pd.read_csv("./data/test.csv")
X_y_train = xgb.DMatrix(data=train[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']], label= train['Survived'])
X_test    = xgb.DMatrix(data=test[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']])

In [ ]:
train[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch', 'Survived']].head()


In [ ]:
params = {
          'base_score': np.mean(train['Survived']),
          'eta':  0.1,
          'max_depth': 3,
          'gamma' :3,
          'objective'   :'reg:linear',
          'eval_metric' :'mae'
         }
model = xgb.train(params=params, 
                  dtrain=X_y_train, 
                  num_boost_round=3)

In [ ]:
model.get_dump()

In [ ]:

y_test=  model.predict(X_test)   
test['pred'] = y_test
test[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch','pred']].iloc[10:].head(2)


# Export model for production with BentoML

In [ ]:
%%writefile xgboost_titanic_model.py

import xgboost as xgb
import bentoml
from bentoml.artifact import XgboostModelArtifact
from bentoml.handlers import DataframeHandler

@bentoml.artifacts([XgboostModelArtifact('model')])
@bentoml.env(conda_dependencies=['xgboost', 'pandas'])
class TitanicModel(bentoml.BentoService):
    
    @bentoml.api(DataframeHandler)
    def predict(self, df):
        data = xgb.DMatrix(data=df[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']])
        return self.artifacts.model.predict(data)

In [ ]:
from xgboost_titanic_model import TitanicModel

bento_model = TitanicModel.pack(
    model = model
)

# Save bento model to a directory
saved_path = bento_model.save('/tmp/bento')

print(saved_path)

# Load model from archive

In [ ]:
import bentoml

bento_model = bentoml.load(saved_path)

result = bento_model.predict(test)
test['pred'] = result
test[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch','pred']].iloc[10:].head(2)